# Carnival Demo

In [2]:
import serial

['CR', 'EIGHTBITS', 'FIVEBITS', 'LF', 'PARITY_EVEN', 'PARITY_MARK', 'PARITY_NAMES', 'PARITY_NONE', 'PARITY_ODD', 'PARITY_SPACE', 'PortNotOpenError', 'SEVENBITS', 'SIXBITS', 'STOPBITS_ONE', 'STOPBITS_ONE_POINT_FIVE', 'STOPBITS_TWO', 'Serial', 'SerialBase', 'SerialException', 'SerialTimeoutException', 'Timeout', 'VERSION', 'XOFF', 'XON', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'absolute_import', 'basestring', 'importlib', 'io', 'iterbytes', 'os', 'protocol_handler_packages', 'serial_for_url', 'serialutil', 'serialwin32', 'sys', 'time', 'to_bytes', 'unicode', 'win32']


In [ ]:
# Maintain a map of distance co-ordinate from sensor to GPS value

In [60]:
def read_data(ser, buf='', callback=None):
    if callback is None:
        callback = print

    # # Read enough data for a message
    # buf += ser.read(ser.in_waiting) # If you are using threading +10 or something so the thread has to wait for more data, this makes the thread sleep and allows the main thread to run.
    # while "[" not in buf or "]" not in buf:
    #     buf += ser.read(ser.in_waiting)

    # # There may be multiple messages received
    # while "[" in buf and ']' in buf:
    #     # Find the message
    #     start = buf.find('[')
    #     buf = buf[start+1:]
    #     end = buf.find(b']')
    #     msg_parts = buf[:end].split(";") # buf now has b"Hello, 1234"
    #     buf = buf[end+1:]

    #     # Check the checksum to make sure the data is valid
    #     print(msg_parts)
    if ser.in_waiting > 0:
        raw_data = ser.readline().strip()
        raw_data.decode('latin1')
        #buf = ser.readline().decode('utf-8').rstrip()
        buf = str(raw_data).rstrip("\n")
        #print(buf)
        start = buf.find('[')
        #print(start)
        buf = buf[start+1:]
        end = buf.find(']')
        #print(end)
        msg_parts = buf[:end].split(";") # buf now has b"Hello, 1234"
        print(msg_parts)
        #return buf



In [63]:
ser.close() 

In [62]:
running = True
ser = serial.Serial("COM7", 9600)
buf = ''
while running:
    buf = read_data(ser, buf)

2
9
4.05
0.00
2
10
4.05
41.42
2
10
4.05
41.54
2
10
4.05
41.52
2
10
4.05
41.42
2
10
3.93
41.54
2
10
4.05
41.42
2
10
4.05
41.52
2
10
4.05
41.54
2
10
4.05
41.42
2
10
3.93
41.52
2
10
4.05
41.42
2
10
4.05
41.54
2
10
4.03
41.52
2
10
4.03
41.88
2
10
3.94
41.52
2
10
4.05
41.43
2
10
4.05
41.52
2
10
4.03
41.98
2
10
4.05
41.88
2
10
3.93
41.52
2
10
4.05
41.42
2
10
4.05
41.52
2
10
4.05
41.52
2
10
4.05
41.88
2
10
3.93
41.52
2
10
4.05
41.86
2
10
4.05
41.98
2
10
4.05
41.52
2
10
4.03
41.42
2
10
3.94
41.52
2
10
4.05
41.42
2
10
4.05
41.52


KeyboardInterrupt: 

In [55]:
ser.close() 

In [16]:
print(serial.__version__)

3.5


In [17]:
print(ser.in_waiting)

12089
